<h1>Looping Graph (Cycles)</h1>

### Intro
Not all graphs are straight lines. Sometimes, you need to **Loop**.
Example: "Keep trying this API until it works," or "Generate numbers until we have 5 of them."

**Goal:** Build a graph that generates random numbers and loops back to itself until a condition is met (5 numbers found).

### Step 1: Imports
Standard imports + `random` for our logic.

In [ ]:
from langgraph.graph import StateGraph, START, END
import random
from typing import Dict, List, TypedDict

### Step 2: The State
We need a `number` list to store our finds, and a `counter` to track the loop.

In [ ]:
class AgentState(TypedDict):
    name: str           # Just for saying hi
    number: List[int]   # We will append random numbers here
    counter: int        # Tracks how many numbers we found

### Step 3: The Nodes
**1. Greeting Node**: Logic to run once at the start.
**2. Random Node**: The worker that will be looped over.

In [ ]:
def greetingNode(state: AgentState) -> AgentState:
    """Greeting node which says hi to the patient""" 
    print("--- Executing Greeting Node ---")
    state["name"] = f"Hi There, {state["name"]}"
    state["counter"] = 0 # initializing counter is necessary so we know where to start
    return state

In [ ]:
def randomNode(state: AgentState) -> AgentState:
    """Generates a random number from 0 to 10"""
    print("--- Executing Random Number Generator ---")
    
    # Generate a random integer
    num = random.randint(0, 10)
    
    # Append it to the list in our state
    state["number"].append(num)
    
    # INCREMENT THE COUNTER! Very important for loops.
    state["counter"] += 1
    
    print(f"Found number: {num}. Total count: {state['counter']}")
    return state

### Step 4: The Decider (Router)
This determines if we stay in the loop or exit.
- If `counter < 5`: Return "loopEdge" (Go back to `randomNode`).
- Else: Return "exitEdge" (Go to `END`).

In [ ]:
def shouldContinue(state: AgentState) -> str:
    """Decider function"""
    # Check our condition
    if(state["counter"] < 5):
        print("-> Looping back...")
        return "loopEdge"
    else:
        print("-> Exiting...")
        return "exitEdge"
     

### Step 5: Build Graph with Cycles
Connecting nodes to themselves (or previous nodes) creates a cycle.

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("greeter", greetingNode)
graph.add_node("random", randomNode)

# Linear flow at start
graph.add_edge(START, "greeter")
graph.add_edge("greeter", "random")

# Conditional flow for the loop
graph.add_conditional_edges(
    "random",          # After randomNode runs...
    shouldContinue,    # Check this logic...
    {
        # If logic returns this : Go here
        "loopEdge": "random", # IMPORTANT: Points BACK to 'random' node
        "exitEdge": END       # Points to finish
    }
)

app = graph.compile()

### Step 6: Visualize
Look closely at the arrow pointing from `random` back to `random`. That is your loop.

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

### Step 7: Run the Loop
We start with an empty list. The graph should run 5 times and fill it.

In [ ]:
input_data = {
    "name": "Debu", 
    "number": [], 
    "counter": 0
}

result = app.invoke(input_data)
print(f"Final State: {result}")